<a href="https://colab.research.google.com/github/MaxGhenis/random/blob/master/wealth_tax_ubi_inequality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Effects of wealth tax and UBI on wealth inequality

Compare a wealth tax to a UBI on wealth inequality.

## Setup

TODO: Figure out how to make microdf work without installing labellines and taxcalc (optional).

In [0]:
!pip install matplotlib-label-lines
!pip install git+git://github.com/maxghenis/microdf.git
!pip install git+git://github.com/PSLmodels/Tax-Calculator.git

In [0]:
import pandas as pd
import numpy as np
import microdf as mdf

## Load data

From Saez/Zucman wealth tax calculator: https://github.com/BITSS/opa-wealthtax

In [0]:
wealth = pd.read_stata('https://github.com/BITSS/opa-wealthtax/blob/master/analysis_data/wealth.dta?raw=true')

## Preprocess

### UBI

Andrew Yang's Freedom Dividend of $1,000 per month for each adult citizen.

Other features like tax revenue and benefit clawback would require fusing with CPS data, so these are not modeled here.

From https://www.census.gov/quickfacts/fact/table/US/PST045218 and https://www.kff.org/other/state-indicator/distribution-by-citizenship-status.

In [0]:
ADULT_POP = 327_167_434 * (1 - 0.224)
PCT_CITIZENS = 0.93
citizen_adults_per_family = PCT_CITIZENS * ADULT_POP / wealth.weight.sum()
UBI_PER_ADULT_CITIZEN = 12000
ubi_per_family = UBI_PER_ADULT_CITIZEN * citizen_adults_per_family

In [0]:
wealth['yang_ubi'] = ubi_per_family
wealth['networth_ubi'] = wealth.networth + wealth.yang_ubi

Total impact ($B)

In [6]:
(mdf.weighted_sum(wealth, 'networth_ubi', 'weight') - 
 mdf.weighted_sum(wealth, 'networth', 'weight')).sum() / 1e9

2833.3223252294374

### Wealth tax

16% avoidance/evasion rate per Saez and Zucman: http://wealthtaxsimulator.org/analysis/

In [0]:
PCT_AVOID_EVADE = 0.16

In [0]:
from bisect import bisect

def tax(income, rates, brackets, base_tax):
    i = bisect(brackets, income)
    if not i:
        return 0
    rate = rates[i]
    bracket = brackets[i-1]
    income_in_bracket = income - bracket
    tax_in_bracket = income_in_bracket * rate
    total_tax = base_tax[i-1] + tax_in_bracket
    return total_tax

In [0]:
def tax_multiple(incomes, **kwargs):
    return [tax(income, **kwargs) for income in incomes]

#### Warren

2% above \$50 million, 3% above \$1 billion, per http://wealthtaxsimulator.org/analysis/



In [0]:
WARREN_RATES = [0, 0.02, 0.03]   # 0%, 2%, 3%.

WARREN_BRACKETS = [50e6,  # First $50 million.
                   1e9]   # Over $1 billion.

WARREN_BASE_TAX = [0,     # $50,000,000 * 0%
                   20e6]  # $1,000,000,000 * 2%

In [0]:
def warren_tax_multiple(incomes):
    return tax_multiple(incomes, rates=WARREN_RATES, brackets=WARREN_BRACKETS,
                        base_tax=WARREN_BASE_TAX)

Test a couple values.

In [12]:
warren_tax_multiple([-1e6,    # Expected: 0
                     0,       # Expected: 0
                     1e6,     # Expected: 0
                     50e6,    # Expected: 0
                     50e6+1,  # Expected: 0.02
                     100e6,   # Expected: 2% * (100e6 - 50e6) = 2% * 50e6 = 1e6
                     1e9,     # Expected: 20,000,000
                     1e9+1,   # Expected: 20,000,000.03
                     10e9     # Expected: 20e6 + 3% * (10e9 - 1e9) = 
                              #           20e6 + 270e6 = 290e6
                     ])

[0, 0, 0, 0.0, 0.02, 1000000.0, 20000000.0, 20000000.03, 290000000.0]

In [0]:
wealth['warren_theoretical_tax'] = tax_multiple(
    wealth.networth, rates=WARREN_RATES, brackets=WARREN_BRACKETS,
    base_tax=WARREN_BASE_TAX)

In [0]:
wealth['warren_tax'] = wealth.warren_theoretical_tax * (1 - PCT_AVOID_EVADE)

In [0]:
wealth['networth_warren'] = wealth.networth - wealth.warren_tax

Total impact ($B)

This differs from the $199 billion estimated for 2019 in http://wealthtaxsimulator.org/analysis/ - is this because this is 2016? Would expect it to be higher in 2019 than 2016.

In [16]:
(mdf.weighted_sum(wealth, 'networth_warren', 'weight') - 
 mdf.weighted_sum(wealth, 'networth', 'weight')).sum() / 1e9

-215.04780760723438

In [17]:
wealth.head()

,data,networth,weight,yang_ubi,networth_ubi,warren_theoretical_tax,warren_tax,networth_warren
0,FB400,1.600000e+11,1.0,18111.728182,1.600000e+11,4.790000e+09,4.023600e+09,1.559764e+11
1,FB400,9.700000e+10,1.0,18111.728182,9.700002e+10,2.900000e+09,2.436000e+09,9.456400e+10
2,FB400,8.830000e+10,1.0,18111.728182,8.830002e+10,2.639000e+09,2.216760e+09,8.608324e+10
3,FB400,6.100000e+10,1.0,18111.728182,6.100002e+10,1.820000e+09,1.528800e+09,5.947120e+10
4,FB400,5.840000e+10,1.0,18111.728182,5.840002e+10,1.742000e+09,1.463280e+09,5.693672e+10


#### Sanders (TODO)

>Sanders wants to levy a 1 percent tax on wealth above \$32 million, for married couples, and then slowly increase the tax for wealthier households: a 2 percent for wealth between \$50 to \$250 million; 3 percent for wealth from \$250 to $500 million; 4 percent from \$500 million to \$1 billion, 5 percent from \$1 to \$2.5 billion, 6 percent from \$2.5 to \$5 billion, 7 percent from \$5 to \$10 billion, and 8 percent on wealth over \$10 billion. Same thing goes for super-rich single people, except the wealth thresholds are cut in half. In other words, an unmarried person with \$16.5 million in wealth would pay a \$5,000 tax, as would a married couple with \$32.5 million in net worth.

https://www.vox.com/policy-and-politics/2019/9/24/20880941/bernie-sanders-wealth-tax-warren-2020

## Analysis

### Gini

In [18]:
gini_base = mdf.gini(wealth.networth, wealth.weight)
gini_base

0.87092919537617

In [19]:
gini_warren = mdf.gini(wealth.networth_warren, wealth.weight)
gini_warren

0.8706327269354546

In [20]:
gini_ubi = mdf.gini(wealth.networth_ubi, wealth.weight)
gini_ubi

0.8453790740253787

In [21]:
gini_warren_chg = gini_base - gini_warren
gini_warren_chg

0.0002964684407154028

In [22]:
gini_ubi_chg = gini_base - gini_ubi
gini_ubi_chg

0.025550121350791266

How much more does the UBI move Gini compared to the Warren tax?

In [23]:
gini_ubi_chg / gini_warren_chg

86.18158914026976

### Share of wealth held by top 1%

In [0]:
mdf.add_weighted_quantiles(wealth, 'networth', 'weight')
mdf.add_weighted_quantiles(wealth, 'networth_warren', 'weight')
mdf.add_weighted_quantiles(wealth, 'networth_ubi', 'weight')

In [0]:
def top_1pct_share(df, percentile_col, wealth_col, weight_col):
    top_1pct_sum = (df[df[percentile_col] > 99][wealth_col] *
                    df[df[percentile_col] > 99][weight_col]).sum()
    total_sum = mdf.weighted_sum(df, wealth_col, weight_col)
    return top_1pct_sum / total_sum

In [26]:
top_1pct_share_base = top_1pct_share(
    wealth, 'networth_percentile_exact', 'networth', 'weight')
top_1pct_share_base

0.4175010487580551

In [27]:
top_1pct_share_warren = top_1pct_share(
    wealth, 'networth_warren_percentile_exact', 'networth_warren', 'weight')
top_1pct_share_warren

0.4161617636383938

In [28]:
top_1pct_share_ubi = top_1pct_share(
    wealth, 'networth_ubi_percentile_exact', 'networth_ubi', 'weight')
top_1pct_share_ubi

0.40554675621572595

In [29]:
top_1pct_share_warren_chg = top_1pct_share_warren - top_1pct_share_base
top_1pct_share_warren_chg

-0.0013392851196613043

In [30]:
top_1pct_share_ubi_chg = top_1pct_share_ubi - top_1pct_share_base
top_1pct_share_ubi_chg

-0.011954292542329159

In [31]:
top_1pct_share_ubi_chg / top_1pct_share_warren_chg

8.925875727904986